In [1]:
#Data handling
import pandas as pd
import numpy as np

#Data preprocessing
import os
import pickle
# from keras.models import load_model
import joblib

#Data modeling
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix, classification_report, roc_curve, roc_auc_score, precision_score, recall_score, f1_score, auc
from sklearn.pipeline import Pipeline
from keras.models import load_model
import warnings

from imblearn.over_sampling import RandomOverSampler, SMOTE

warnings.filterwarnings('ignore')


In [2]:
input_features =[
    'Date',
    'Location',
    'MinTemp',
    'MaxTemp',
    'Rainfall',
    'Evaporation',
    'Sunshine',
    'WindGustDir',
    'WindGustSpeed',
    'WindDir9am',
    'WindDir3pm',
    'WindSpeed9am',
    'WindSpeed3pm',
    'Humidity9am',
    'Humidity3pm',
    'Pressure9am',
    'Pressure3pm',
    'Cloud9am',
    'Cloud3pm',
    'Temp9am',
    'Temp3pm',
    'RainToday',
    'RainTomorrow',
    'RainfallTomorrow'
    ]

selected_features = [
                'MinTemp', 
                'MaxTemp', 
                'Evaporation',
                'Sunshine',
                'Wind9amu', 
                'Wind9amv', 
                'Wind3pmu', 
                'Wind3pmv', 
                'WindGustu',
                'WindGustv', 
                'Max_Pressure',
                'Min_Pressure',
                'Max_Humidity',
                'Min_Humidity',
                'Estado_Cielo',
                'Cluster',
                'Date'
            ]

In [3]:
def feat_eng_regr(df):
    def cluster_cities(df):
        city_cluster = {
            'Adelaide': 10,
            'Albany': 1,
            'Albury': 4,
            'AliceSprings': 3,
            'BadgerysCreek': 2,
            'Ballarat': 8,
            'Bendigo': 8,
            'Brisbane': 9,
            'Cairns': 6,
            'Canberra': 4,
            'Cobar': 4,
            'CoffsHarbour': 9,
            'Dartmoor': 0,
            'Darwin': 7,
            'GoldCoast': 9,
            'Hobart': 12,
            'Katherine': 7,
            'Launceston': 12,
            'Melbourne': 8,
            'MelbourneAirport': 8,
            'Mildura': 0,
            'Moree': 9,
            'MountGambier': 0,
            'MountGinini': 4,
            'Newcastle': 2,
            'Nhil': 0,
            'NorahHead': 2,
            'NorfolkIsland': 5,
            'Nuriootpa': 10,
            'PearceRAAF': 1,
            'Penrith': 2,
            'Perth': 1,
            'PerthAirport': 1,
            'Portland': 0,
            'Richmond': 2,
            'Sale': 8,
            'Sydney': 2,
            'SydneyAirport': 2,
            'Townsville': 6,
            'Tuggeranong': 4,
            'Uluru': 3,
            'WaggaWagga': 4,
            'Walpole': 1,
            'Watsonia': 8,
            'Williamtown': 2,
            'Witchcliffe': 1,
            'Wollongong': 2,
            'Woomera': 10
            }
    
        cluster = df['Location'].map(city_cluster)
    
        return cluster
    

    def vector_coordinates(df, column, dir, speed, threshold=1e-6):
        direcciones = {
            'N': 0,
            'NNE': 22.5,
            'NE': 45,
            'ENE': 67.5,
            'E': 90,
            'ESE': 112.5,
            'SE': 135,
            'SSE': 157.5,
            'S': 180,
            'SSW': 202.5,
            'SW': 225,
            'WSW': 247.5,
            'W': 270,
            'WNW': 292.5,
            'NW': 315,
            'NNW': 337.5
        }
        velocidad = df[speed]
        df[column + 'u'] = -velocidad * np.sin(np.deg2rad(df[dir].map(direcciones)))
        df[column + 'v'] = -velocidad * np.cos(np.deg2rad(df[dir].map(direcciones)))
        
        # Comprobación condicional para valores cercanos a cero
        df[column + 'u'][np.abs(df[column + 'u']) < threshold] = 0
        df[column + 'v'][np.abs(df[column + 'v']) < threshold] = 0
        
        return df

    def definir_estado_del_cielo(row):
        if row['Cloud9am'] == 0 and row['Cloud3pm'] == 0:
            return 'Despejado'
        elif row['Cloud9am'] <= 4 and row['Cloud3pm'] <= 4:
            return 'Parcialmente Nublado'
        elif row['Cloud9am'] <= 7 and row['Cloud3pm'] <= 7:
            return 'Nublado'
        else:
            return 'Muy Nublado'
    
    df=vector_coordinates(df,'Wind9am','WindDir9am','WindSpeed9am')
    df=vector_coordinates(df,'Wind3pm','WindDir3pm','WindSpeed3pm')
    df=vector_coordinates(df,'WindGust','WindGustDir','WindGustSpeed')
    df['Max_Pressure'] = df[['Pressure9am', 'Pressure3pm']].max(axis=1)
    df['Min_Pressure'] = df[['Pressure9am', 'Pressure3pm']].min(axis=1)
    df['Max_Humidity'] = df[['Humidity9am', 'Humidity3pm']].max(axis=1)
    df['Min_Humidity'] = df[['Humidity9am', 'Humidity3pm']].min(axis=1)
    df['Estado_Cielo'] = df.apply(definir_estado_del_cielo, axis=1)
    df['Cluster'] = cluster_cities(df)

    df.drop(['Cloud3pm','Cloud9am','Humidity3pm','Humidity9am','WindSpeed9am','WindSpeed3pm','WindGustSpeed','WindDir9am','WindDir3pm','WindGustDir','Pressure3pm','Pressure9am','Temp3pm','Temp9am'], axis=1, inplace=True)
    df = df.replace([np.inf, -np.inf], 0)

    return df

In [4]:
def feat_eng_clas(df):
    df_normalized = df.copy()
    df_normalized['Min_Humidity'] = df_normalized[['Humidity9am', 'Humidity3pm']].min(axis=1)

    df_normalized.drop(['Humidity3pm','Humidity9am'], axis=1, inplace=True)
    df_normalized = df_normalized.replace([np.inf, -np.inf], 0)

    scaler = StandardScaler()
    df_normalized = pd.DataFrame(scaler.fit_transform(df_normalized), columns=df_normalized.columns)

    X = df_normalized[['Sunshine','Min_Humidity']]

    return X

In [5]:
def dataframe_normalized_regress(df,features):
    df_normalized = df.copy()
    skystate = {
        'Despejado' :               0,
        'Parcialmente Nublado' :    1,
        'Nublado' :                 2,
        'Muy Nublado' :             3
    }
    df_normalized['RainToday'] = df_normalized['RainToday'].map({'No': 0, 'Yes': 1})
    df_normalized['RainTomorrow'] = df_normalized['RainTomorrow'].map({'No': 0, 'Yes': 1})
    df_normalized['Date'] = pd.to_datetime(df_normalized['Date'])
    df_normalized['Date']=df_normalized['Date'].dt.dayofyear
    df_normalized['Estado_Cielo'] = df_normalized['Estado_Cielo'].map(skystate)
    scaler = StandardScaler()
    df_normalized[features] = scaler.fit_transform(df_normalized[features])
    
    columns =   [
                    'MinTemp', 
                    'MaxTemp', 
                    'Evaporation',
                    'Sunshine',
                    'Wind9amu', 
                    'Wind9amv', 
                    'Wind3pmu', 
                    'Wind3pmv', 
                    'WindGustu',
                    'WindGustv', 
                    'Max_Pressure',
                    'Min_Pressure',
                    'Max_Humidity',
                    'Min_Humidity',
                    'Estado_Cielo',
                    'Cluster',
                    'Date'
                ]

    X = df_normalized[columns]

    return X

In [6]:
clasificadorNN = load_model('ClasificadorNN.h5')

In [7]:
pipeline_clasf = Pipeline([
    ('Feature Engineering', FunctionTransformer(feat_eng_clas)),
    ('Model', clasificadorNN)
])


In [8]:
regresionNN = load_model('RegresionNN.h5')

In [9]:
pipeline_regress = Pipeline([
    ('Feature Engineering', FunctionTransformer(feat_eng_regr)),
    ('df Normalized', FunctionTransformer(dataframe_normalized_regress, kw_args={'features': selected_features})),
    ('Model', regresionNN)
])

In [10]:
df = pd.read_csv('./files/weatherAUS_sin_nan.csv', usecols=input_features)

In [11]:
#I want to obtain only a sample of the data to use for the prediction and check wether the pipeline works
df_sample = df.sample(1, random_state=42)

In [12]:
#i want the values of the sample to be in a dict
df_sample_dict = df_sample.to_dict('records')
df_sample_dict

[{'Date': '2009-08-01',
  'Location': 'NorfolkIsland',
  'MinTemp': 11.3,
  'MaxTemp': 18.4,
  'Rainfall': 0.0,
  'Evaporation': 2.0,
  'Sunshine': 8.7,
  'WindGustDir': 'WSW',
  'WindGustSpeed': 19.0,
  'WindDir9am': 'ENE',
  'WindDir3pm': 'WSW',
  'WindSpeed9am': 2.0,
  'WindSpeed3pm': 11.0,
  'Humidity9am': 70.0,
  'Humidity3pm': 61.0,
  'Pressure9am': 1022.0,
  'Pressure3pm': 1019.6,
  'Cloud9am': 1.0,
  'Cloud3pm': 1.0,
  'Temp9am': 17.5,
  'Temp3pm': 18.0,
  'RainToday': 'No',
  'RainTomorrow': 'No',
  'RainfallTomorrow': 0.0}]

In [13]:
print(df_sample.columns)

Index(['Date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
       'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'RainTomorrow', 'RainfallTomorrow'],
      dtype='object')


In [14]:
pipeline_clasf

Pipeline(steps=[('Feature Engineering',
                 FunctionTransformer(func=<function feat_eng_clas at 0x00000264AF4D6E60>)),
                ('Model',
                 <keras.engine.sequential.Sequential object at 0x00000264E8004A00>)])

In [15]:
#I want to check if the pipeline works predicting the sample
# pipeline.fit(df_sample)
a = pipeline_clasf.predict(df_sample[['Sunshine', 'Humidity9am', 'Humidity3pm']])
a[0][0]

1/1 [==============================] - 2s 2s/step


0.5670569

In [16]:
joblib.dump(pipeline_clasf, 'rain_or_not_prediction.joblib')

INFO:tensorflow:Assets written to: ram://627dcd2b-09a7-4c42-b2a5-70a9ea9f965d/assets


['rain_or_not_prediction.joblib']

In [17]:
pipeline_regress

Pipeline(steps=[('Feature Engineering',
                 FunctionTransformer(func=<function feat_eng_regr at 0x00000264AF4D69E0>)),
                ('df Normalized',
                 FunctionTransformer(func=<function dataframe_normalized_regress at 0x00000264E7F4D000>,
                                     kw_args={'features': ['MinTemp', 'MaxTemp',
                                                           'Evaporation',
                                                           'Sunshine',
                                                           'Wind9amu',
                                                           'Wind9amv',
                                                           'Wind3pmu',
                                                           'Wind3pmv',
                                                           'WindGustu',
                                                           'WindGustv',
                                                           'Max_Pressure',
                                                           'Min_Pressure',
                                                           'Max_Humidity',
                                                           'Min_Humidity',
                                                           'Estado_Cielo',
                                                           'Cluster',
                                                           'Date']})),
                ('Model',
                 <keras.engine.sequential.Sequential object at 0x00000264E8006A70>)])

In [18]:
b = pipeline_regress.predict(df_sample)
b[0][0]

1/1 [==============================] - 0s 49ms/step


0.41641378

In [19]:
joblib.dump(pipeline_regress, 'rainfall_amount_prediction.joblib')

INFO:tensorflow:Assets written to: ram://b1572b82-77c3-432a-9a39-d09642723fd6/assets


['rainfall_amount_prediction.joblib']